# New notebook

In [59]:
import datetime

import pandas as pd
import sqlite3 as db
from sqlalchemy import create_engine
db_name = "test2.db"
engine = create_engine(f"sqlite:///{db_name}")

In [46]:
symbols_data = [
        {
            "Symbol": "A",
            "Name": "Agilent Technologies Inc",
            "ListedDt": datetime.datetime(2005, 1, 3).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "AA",
            "Name": "Alcoa Corporation",
            "ListedDt": datetime.datetime(2016, 10, 18).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "ZGNX",
            "Name": "Zogenix",
            "ListedDt": datetime.datetime(2010, 11, 23).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        }
    ]

update_data = [
        {
            "Symbol": "AA",
            "Name": "Updated Alcoa Corporation",
            "ListedDt": datetime.datetime(2022, 10, 18).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "ZGNX",
            "Name": "Updated Zogenix",
            "ListedDt": datetime.datetime(2012, 11, 23).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "BC",
            "Name": "Basic Company",
            "ListedDt": datetime.datetime(2021, 1, 1).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        }
    ]

In [47]:
source = pd.DataFrame(symbols_data)
new_data = pd.DataFrame(update_data)

In [49]:
suffixe_new = '_new'
cols_old = source.columns
cols_new = [f"{col}{suffixe_new}" for col in cols_old]
cols_new[0] = cols_old[0]

In [51]:
cols_old

Index(['Symbol', 'Name', 'ListedDt', 'LastDt', 'Status'], dtype='object')

In [52]:
cols_new

['Symbol', 'Name_new', 'ListedDt_new', 'LastDt_new', 'Status_new']

In [53]:
merged = pd.merge(source, new_data, on='Symbol', how='outer', indicator=True, suffixes=['', '_new'])
merged

,Symbol,Name,ListedDt,LastDt,Status,Name_new,ListedDt_new,LastDt_new,Status_new,_merge
0,A,Agilent Technologies Inc,2005-01-03T00:00:00,2022-09-06T00:00:00,Active,NaN,NaN,NaN,NaN,left_only
1,AA,Alcoa Corporation,2016-10-18T00:00:00,2022-09-06T00:00:00,Active,Updated Alcoa Corporation,2022-10-18T00:00:00,2022-09-06T00:00:00,Active,both
2,ZGNX,Zogenix,2010-11-23T00:00:00,2022-03-04T00:00:00,Active,Updated Zogenix,2012-11-23T00:00:00,2022-03-04T00:00:00,Active,both
3,BC,NaN,NaN,NaN,NaN,Basic Company,2021-01-01T00:00:00,2022-03-04T00:00:00,Active,right_only


In [63]:
old = merged.loc[merged._merge == 'left_only'][cols_old]
old

,Symbol,Name,ListedDt,LastDt,Status
0,A,Agilent Technologies Inc,2005-01-03T00:00:00,2022-09-06T00:00:00,Active


In [68]:
updates = merged.loc[merged._merge == 'both'][cols_new]
updates.columns = cols_old
updates

,Symbol,Name,ListedDt,LastDt,Status
1,AA,Updated Alcoa Corporation,2022-10-18T00:00:00,2022-09-06T00:00:00,Active
2,ZGNX,Updated Zogenix,2012-11-23T00:00:00,2022-03-04T00:00:00,Active


In [69]:
new_sym = merged.loc[merged._merge == 'right_only'][cols_new]
new_sym.columns = cols_old
new_sym

,Symbol,Name,ListedDt,LastDt,Status
3,BC,Basic Company,2021-01-01T00:00:00,2022-03-04T00:00:00,Active


In [70]:
db_update = pd.concat([old, updates, new_sym], ignore_index=True)
db_update

,Symbol,Name,ListedDt,LastDt,Status
0,A,Agilent Technologies Inc,2005-01-03T00:00:00,2022-09-06T00:00:00,Active
1,AA,Updated Alcoa Corporation,2022-10-18T00:00:00,2022-09-06T00:00:00,Active
2,ZGNX,Updated Zogenix,2012-11-23T00:00:00,2022-03-04T00:00:00,Active
3,BC,Basic Company,2021-01-01T00:00:00,2022-03-04T00:00:00,Active


In [72]:
old.to_sql('Symbols', engine, if_exists='replace', index=False)
updates.to_sql('Symbols', engine, if_exists='append', index=False)
new_sym.to_sql('Symbols', engine, if_exists='append', index=False)


1